In [22]:

import json
import pandas as pd
from pathlib import Path
from common.utils.utils import estimate_solution_score
from common.models.network import Network

In [10]:
networks_path = Path("../data/23_11_30")
network_data = json.load(open(networks_path / "networks.json"))

solutions = {
    "myopic": json.load(open(networks_path / "solution__myopic.json")),
    "m1": json.load(open(networks_path / "solutions" / "0.json")),
    "random": json.load(open(networks_path / "solution__random.json")),
    "take_loss": json.load(open(networks_path / "solution__take_loss.json")),
}

def key_solutions(solutions):
    return {s["network_id"]: s for s in solutions}

solutions_by_network_id = {k: key_solutions(v) for k, v in solutions.items()}

In [23]:
scores = []
for network in network_data:
    n = Network(**network)
    n_id = n.network_id
    for s_type in solutions.keys():
        solution = solutions_by_network_id[s_type][n_id]
        score = estimate_solution_score(n, moves=solution['moves'], n_steps=10)
        scores.append({
            "network_id": n_id,
            "type": s_type,
            "score": score
        })

scores_df = pd.DataFrame(scores)

In [29]:
scores_df = scores_df.pivot_table(index="network_id", columns="type", values="score")

In [31]:
scores_df['m1-myopic'] = scores_df['m1'] - scores_df['myopic']
scores_df['m1-random'] = scores_df['m1'] - scores_df['random']
scores_df['m1-take_loss'] = scores_df['m1'] - scores_df['take_loss']

In [35]:
(scores_df['m1-myopic'] < 0).mean()

0.1

In [36]:
(scores_df['m1-take_loss'] < 0).mean()

0.07

In [37]:
scores_df.mean()

type
m1              2450.0
myopic          1575.0
random           790.0
take_loss       2409.5
m1-myopic        875.0
m1-random       1660.0
m1-take_loss      40.5
dtype: float64